In [26]:
import torch
torch.__version__

'1.10.2+cu102'

In [27]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import numpy as np
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from tqdm.auto import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc#my texts

test = pd.read_csv('data/HeadHunter_test_prep.csv')
train = pd.read_csv('data/HeadHunter_train_prep.csv')

In [28]:
y_train = train['target']
y_tr = []
for labels in list(y_train):
    y_tr.append([int(j) for j in labels.split(',')])

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_train_multilabels = mlb.fit_transform(list(y_tr))

df_ = pd.DataFrame(list(train['review_id']), columns = ['review_id'])
df_['comment_text'] = train['positive'] + ' | ' + train['negative']#train['position'] + ' | ' + 
for i in ['0', '1', '2', '3', '4', '5', '6', '7', '8']:
    df_[i] = y_train_multilabels[:, int(i)].astype(float)
df_train = df_

df_ = pd.DataFrame(list(test['review_id']), columns = ['review_id'])
df_['comment_text'] = test['positive'] + ' | ' + test['negative']#test['position'] + ' | ' + 
df_test = df_

In [29]:
features = ['salary_rating', 'team_rating', 'managment_rating', 'career_rating', 'workplace_rating',
            'rest_recovery_rating', 'population', 'position_len', 'negative_len',
            'positive_len', 'edit_d', 'pos_the_longest_token',
            'neg_the_longest_token', 'pos_lex_div', 'neg_lex_div']
train_features = train[features].values.astype(np.float32)
# test# train_features = train[features].values.astype(np.float32)
test_features = test[features].values.astype(np.float32)

In [30]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
train_features = ss.fit_transform(train_features)
test_features = ss.fit_transform(test_features)
df_train.fillna('', inplace=True)
df_test.fillna('', inplace=True)

In [32]:
from transformers import BertTokenizer, BertForSequenceClassification

In [47]:
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup

import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):

  def __init__(self, texts, features, targets, tokenizer, max_len=None):
    self.texts = texts
    self.features = features
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    text = str(self.texts[idx])
    target = self.targets[idx]
    features = self.features[idx]

    encoding = self.tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        truncation=True,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
    )

    return {
      'text': text,
      'features' : features,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target)#torch.tensor(target, dtype=torch.long)
    }
    
from transformers import AutoConfig, AutoModel
import torch

from transformers import BertModel
class CustomBERTModel(nn.Module):
    def __init__(self, bert_model, cfg, features_len, n_out):
        super(CustomBERTModel, self).__init__()
        self.bert_model = bert_model
        self.features_len = features_len
        self.out_features = self.bert_model.config.hidden_size + features_len
        ### New layers:
        self.classifier = nn.Sequential(
          nn.Linear(self.out_features, 64),  
          nn.ReLU(),
          nn.Linear(64, n_out)
        )

    def forward(self, input_ids, attention_mask, features):
        hidden_states = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)

        cls_embeds = hidden_states.last_hidden_state[:, 0, :] 
        concat = torch.cat((cls_embeds, features), dim=-1)
        linear_output = self.classifier(concat)
        return linear_output

class BertClassifier:
    def __init__(self, model_path, features_len, tokenizer_path, MAX_len = 128, n_classes=2, epochs=1, model_save_path='/content/bert.pt'):
        self.config = AutoConfig.from_pretrained(model_path)
        self.bert_model = AutoModel.from_pretrained(model_path, config=self.config)
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model_save_path=model_save_path
        self.max_len = MAX_len
        self.epochs = epochs
        self.model = CustomBERTModel(bert_model=self.bert_model, cfg=self.config, features_len=features_len, n_out=n_classes)
        self.model.to(self.device)
    
    def preparation(self, X_train, X_train_features, y_train, X_valid, X_valid_features, y_valid):
        # create datasets
        self.train_set = CustomDataset(X_train, X_train_features, y_train, self.tokenizer, max_len=self.max_len)
        self.valid_set = CustomDataset(X_valid, X_valid_features, y_valid, self.tokenizer, max_len=self.max_len)

        # create data loaders
        self.train_loader = DataLoader(self.train_set, batch_size=2, shuffle=True)
        self.valid_loader = DataLoader(self.valid_set, batch_size=2, shuffle=True)

        # helpers initialization
        self.optimizer = AdamW(self.model.parameters(), lr=2e-5, correct_bias=False)
        self.scheduler = get_linear_schedule_with_warmup(
                self.optimizer,
                num_warmup_steps=0,
                num_training_steps=len(self.train_loader) * self.epochs
            )
        self.loss_fn = torch.nn.BCEWithLogitsLoss().to(self.device)

    def fit(self):
        self.model = self.model.train()
        losses = []
        correct_predictions = 0
        preds, true = None, None

        for data in tqdm(self.train_loader):
            input_ids = data["input_ids"].to(self.device)
            features = data["features"].to(self.device)
            attention_mask = data["attention_mask"].to(self.device)
            targets = data["targets"].to(self.device)

            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                features=features
                )
            
            if preds is None:
                preds = outputs
                true = targets
            else:
                preds = torch.vstack([preds, outputs])
                true = torch.vstack([true, targets])
            
            loss = self.loss_fn(outputs, targets)
            losses.append(loss.item())
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            self.optimizer.step()
            self.scheduler.step()
            self.optimizer.zero_grad()

        train_acc = f1_score(preds.cpu() > 0.5, true.cpu(), average='samples')
        train_loss = np.mean(losses)
        return train_acc, train_loss
    
    def eval(self):
        self.model = self.model.eval()
        losses = []
        preds, true = None, None

        with torch.no_grad():
            for data in tqdm(self.valid_loader):
                input_ids = data["input_ids"].to(self.device)
                features = data["features"].to(self.device)
                attention_mask = data["attention_mask"].to(self.device)
                targets = data["targets"].to(self.device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    features=features
                    )

                if preds is None:
                    preds = outputs
                    true = targets
                else:
                    preds = torch.vstack([preds, outputs])
                    true = torch.vstack([true, targets])
                
                loss = self.loss_fn(outputs, targets)
                losses.append(loss.item())
        
        val_acc = f1_score(preds.cpu() > 0.5, true.cpu(), average='samples')
        val_loss = np.mean(losses)
        return val_acc, val_loss
    
    def train(self):
        best_accuracy = 0
        for epoch in range(self.epochs):
            print(f'Epoch {epoch + 1}/{self.epochs}')
            train_acc, train_loss = self.fit()
            print(f'Train loss {train_loss} accuracy {train_acc}')

            val_acc, val_loss = self.eval()
            print(f'Val loss {val_loss} accuracy {val_acc}')
            print('-' * 10)
            model_save_path_with_metrics = self.model_save_path.split('.')[0] + '_' + str(round(train_acc, 3)) + '_' + str(round(val_acc, 3)) + '.pt'
            torch.save(self.model, model_save_path_with_metrics)
            if val_acc > best_accuracy:
                torch.save(self.model, self.model_save_path)
                best_accuracy = val_acc

        self.model = torch.load(self.model_save_path)
    
    def predict(self, text, features):
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        out = {
              'text': text,
              'input_ids': encoding['input_ids'].flatten(),
              'attention_mask': encoding['attention_mask'].flatten()
          }
        
        input_ids = out["input_ids"].to(self.device)
        attention_mask = out["attention_mask"].to(self.device)
        
        outputs = self.model(
            input_ids=input_ids.unsqueeze(0),
            attention_mask=attention_mask.unsqueeze(0),
            features=features
        )
        
        prediction = torch.nn.functional.sigmoid(outputs).cpu().detach().numpy()[0]

        return prediction

In [48]:
features_len = len(features)

In [50]:
classifier = BertClassifier(
        model_path = 'cointegrated/rubert-tiny',#модель побольше - 'blanchefort/rubert-base-cased-sentiment',
        tokenizer_path = 'cointegrated/rubert-tiny',#модель побольше -'blanchefort/rubert-base-cased-sentiment',
        MAX_len = 128,    
        n_classes=9,
        epochs=10,
        model_save_path='rubert_tiny.pt',
        features_len = features_len
)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [51]:
data_train, data_valid, f_train, f_valid = train_test_split(df_train, train_features, test_size=0.33, random_state=42)
    #_, data_valid, _, f_valid = train_test_split(data_valid, f_valid, test_size=0.01, random_state=42)

X_train = list(data_train['comment_text'])
y_train = list(data_train[[str(i) for i in range(9)]].values)

X_valid = list(data_valid['comment_text'])
y_valid = list(data_valid[[str(i) for i in range(9)]].values)

classifier.preparation(
    X_train=X_train,
    X_train_features=f_train,
    y_train=y_train,
    X_valid=X_valid,
    X_valid_features=f_valid,
    y_valid=y_valid
)

In [52]:
# torch.save(classifier.model, 'tiny_bert_one_more_attempt.pt')
# classifier.model = torch.load('tiny_bert_one_more_attempt.pt')

In [ ]:
classifier.train()

Epoch 1/10


  0%|          | 0/17043 [00:00<?, ?it/s]

Train loss 0.15114413722924147 accuracy 0.7499970662442059


  0%|          | 0/8395 [00:00<?, ?it/s]

Val loss 0.1363672128138708 accuracy 0.7885447687115346
----------
Epoch 2/10


...

  0%|          | 0/8395 [00:00<?, ?it/s]

Val loss 0.13952822241083335 accuracy 0.7931606114750843
----------
Epoch 3/10


  0%|          | 0/17043 [00:00<?, ?it/s]

Train loss 0.12073350756351273 accuracy 0.811374171213988


  0%|          | 0/8395 [00:00<?, ?it/s]

In [ ]:
# classifier.model = torch.load('add_position_rubert_tiny_0.816_0.97.pt')

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
predictions = np.stack([classifier.predict(txt, f.unsqueeze(0)) for txt, f in tqdm(list(zip(X_train, torch.tensor(f_train).to(device))))])

In [ ]:
def get_score(thresholds):
    y_true = y_train
    y_pred = np.stack([predictions[::, i] > thresholds[i] for i in range(9)]).T.astype(int)
    return f1_score(y_pred, y_true, average='samples')

In [ ]:
!pip install optuna

In [ ]:
import optuna

def objective(trial):
    thresholds = {0: trial.suggest_float('0', 0.0, 1.0),
     1: trial.suggest_float('1', 0.0, 1.0),
     2: trial.suggest_float('2', 0.0, 1.0),
     3: trial.suggest_float('3', 0.0, 1.0),
     4: trial.suggest_float('4', 0.0, 1.0),
     5: trial.suggest_float('5', 0.0, 1.0),
     6: trial.suggest_float('6', 0.0, 1.0),
     7: trial.suggest_float('7', 0.0, 1.0),
     8: trial.suggest_float('8', 0.0, 1.0),}
    return get_score(thresholds)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000)

In [ ]:
thresholds = study.best_params

In [ ]:
thresholds

In [ ]:
# thresholds['0'] = 0.05
# thresholds['8'] = 0.195

In [ ]:
# pd.DataFrame.from_records([thresholds]).to_csv('best_tresholds_tiny_rubert_128_0_0.05_dotrain.csv')

In [ ]:
test_predictions_probs = np.stack([classifier.predict(txt, f.unsqueeze(0)) for txt, f in tqdm(list(zip(df_test.comment_text, torch.tensor(test_features).to(device))))])
test_predictions = np.stack([test_predictions_probs[::, i] > thresholds[str(i)] for i in range(9)]).T.astype(int)

In [ ]:
test_predictions.shape

In [ ]:
# pd.concat([df_test[['review_id']], pd.DataFrame(test_predictions_probs, columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8'])], axis=1).to_csv('rubert_tiny_128_0.823_0.879_0_0.05_probs_test_dotrain.csv')

In [ ]:
from copy import deepcopy
y_pred_test = test_predictions
y_pred_test_mod = deepcopy(y_pred_test)
for i in range(len(y_pred_test_mod)):
    if np.sum(y_pred_test_mod[i]) == 0:
        y_pred_test_mod[i][0] = 1

In [ ]:
example_submit = df_test[['review_id']]#$pd.read_csv('HeadHunter_sample_submit.csv')

target_column = list()
for i in range(len(y_pred_test_mod)):
    target_column.append(','.join(map(str, np.where(y_pred_test_mod[i])[0])))

example_submit['target'] = target_column
example_submit.to_csv("submits/rubert_tiny_128_128_0.825_0.97.csv", index=False)
#"rubert_tiny_128_0.823_0.879_0_0.05_test_labels_dotrain.csv"